# Pregunta 2: Schnorr signature

## Imports

In [93]:
import hashlib
from random import randint

## Helper functions

In [94]:
# Function extracted from T2

def exp_mod(a: int, b: int, n: int) -> int:
    """
    Modular exponentation algorithm

    Args:
        a (int): number >= 0
        b (int): number >= 0
        n (int): number > 0

    Returns:
        int: a**b mod n
    """
    a = a % n
    if n == 1:
        return 0
    if b == 0:
        return 1
    if a == 0:
        return 0
    c = 1
    while b > 0:
        if ((b % 2) != 0):
            c = (c * a) % n
        b //= 2
        a = (a * a) % n
    return c


## Read files

In [95]:
def read_grupo(pathname="grupo.txt"):
    """
    Loads global constants p, g, and q from file

    Arguments:
        pathname (str): path to file

    Returns:
        tuple[int]: values p, g, and q extracted from file.
    """
    with open(pathname, 'r') as grupo_file:
        grupo = grupo_file.readlines()
        p = int(grupo[0].strip(), 16)
        g = int(grupo[1].strip(), 16)
        q = int(grupo[2].strip(), 16)
    return p, g, q

def read_private_key(pathname="private_key.txt"):
    """
    Loads private key from file

    Arguments:
        pathname (str): path to file

    Returns:
        int: private key extracted from file.
    """
    with open(pathname, 'r') as private_key_file:
        private_key = int(private_key_file.readline().strip(), 16)
    return private_key

def read_public_key(pathname="public_key.txt"):
    """
    Loads public key from file

    Arguments:
        pathname (str): path to file

    Returns:
        int: public key extracted from file.
    """
    with open(pathname, 'r') as public_key_file:
        public_key = int(public_key_file.readline().strip(), 16)
    return public_key


## Implementation

In [91]:
def generar_clave_ElGamal():
    """
    Generates a public and private key with ElGamal protocol, with base values extracted from
    grupo.txt file.

    Stores the private key in private_key.txt file and the public key in public_key.txt file
    """
    p, g, q = read_grupo()

    # Generate the private key
    x = randint(1, q - 1)
    y = exp_mod(g, x, p)

    # Output keys
    with open('public_key.txt', 'w') as pk_file:
        pk_file.write(hex(x)[2:].upper())
    with open('private_key.txt', 'w') as sk_file:
        sk_file.write(hex(y)[2:].upper())

def firmar_Schnorr(m: str) -> tuple[int, int]:
    """
    Signs a message with Schnorr protocol, using the private key stored in private_key.txt file, and
    variables stored in grupo.txt file.

    Arguments:
        m (str): Message to be signed
    
    Returns:
        tuple[int, int]: message Schnorr signature in the form of (e, s)
    """
    p, g, q = read_grupo()
    x = read_private_key()
    k = randint(1, q - 1)

    r = exp_mod(g, k, p)
    e = md5(str(r) + m)
    s = (k - x * e) % (p - 1)
    
    return (e, s)

def verificar_firma_Schnorr(m: str, firma: tuple[int, int]) -> bool:
    """
    Verifies if a Schnorr signature corresponds to the specified message, using the public key
    stored in public_key.txt file, and variables stored in grupo.txt file.

    Arguments:
        m (str): Message to be verified
        firma (tuple[int,int]): Signature to be verified

    Returns:
        bool: True if the signature corresponds to the message, False otherwise
    """
    p, g, _ = read_grupo()
    e, s = firma
    y = read_public_key()

    r_prime = (exp_mod(g, s, p) * exp_mod(y, e, p)) % p

    return md5(str(r_prime) + m) == e


def md5(m: str) -> int:
    """
    Hashes a message using md5 function

    Arguments:
        m (str): Message to be hashed

    Returns:
        int: Hash of the message in the form of int
    """
    return int.from_bytes(hashlib.md5(bytes(m, 'utf-8')).digest(), 'big')


## Tests

### Generate key

In [92]:
if __name__ == "__main__":
    # Generate private_key.txt and public_key.txt
    generar_clave_ElGamal()

    # Firmar mensaje
    test_message = "Hello world!"
    firma = firmar_Schnorr(test_message)
    print(firma[0], '\n')
    print(firma[1], '\n')

    # Validar firma
    resultado = verificar_firma_Schnorr(test_message, firma)
    print(resultado)

    

114683320730021126149584038627947937835
3878087035123457967346130535211700548448161168893280679007222504689215963468413375907648803583905051911208795508959011056339069010317285547134310968027481682290950181387280643671615370294984352711763635299562054524355736122807067757692456870862991268397103163296966970224692783845264245030886390072105785228281534179971184120696547334105965658609310918359781430200112752056039432339785501192501117691699109566818448927478030347872374231831889671837178435708054181885981691017609191792856839897640934318789915675743724139003276802025649702433879549324678749248864694602561752140779518891446783712953949743912869551591
False
